In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install pyTelegramBotAPI

     |████████████████████████████████| 92kB 6.0MB/s 
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-3.7.7-cp37-none-any.whl size=60362 sha256=10362042444e3096c683cf2a0480c21e0a3a7da57d443985b908b54b72e14a29
  Stored in directory: /root/.cache/pip/wheels/c8/4d/74/96a36189c5782d9b269404d40c2972f8751b9c27f03a0bc466
Successfully built pyTelegramBotAPI


In [3]:
import re
import telebot
from telebot import apihelper
from tensorflow import keras
import keras.preprocessing.sequence
from keras.preprocessing.sequence import pad_sequences
import pickle

In [4]:
RE_PATTERNS = {
    ' american ':
        [
            'amerikan'
        ],

    ' adolf ':
        [
            'adolf'
        ],


    ' hitler ':
        [
            'hitler'
        ],

      ' fuck ':
      [
        '(f)(u|[^a-z0-9 ])(c|[^a-z0-9 ])(k|[^a-z0-9 ])([^ ])*',
        '(f)([^a-z]*)(u)([^a-z]*)(c)([^a-z]*)(k)',
        ' f[!@#\$%\^\&\*]*u[!@#\$%\^&\*]*k', 'f u u c',
        '(f)(c|[^a-z ])(u|[^a-z ])(k)', r'f\*',
        'feck ', ' fux ', 'f\*\*','fu*k', 'f+u+c+k+',
        'f\-ing', 'f\.u\.', 'f###', ' fu ', 'f@ck', 'f u c k', 'f uck', 'f ck'
      ],

    ' ass ':
        [
            '[^a-z]ass ', '[^a-z]azz ', 'arrse', ' arse ', '@\$\$',
            '[^a-z]anus', ' a\*s\*s', '[^a-z]ass[^a-z ]', 'a+s+s+'
            'a[@#\$%\^&\*][@#\$%\^&\*]', '[^a-z]anal ', 'a s s','a55', '@$$'
        ],

    ' ass hole ':
        [
            ' a[s|z]*wipe', 'a[s|z]*[w]*h[o|0]+[l]*e', '@\$\$hole', 'a*hole', 'a+s+s+h+o+l+e+'
        ],

    ' bitch ':
        [
            'b[w]*i[t]*ch', 'b!tch',
            'bi\+ch', 'b!\+ch', '(b)([^a-z]*)(i)([^a-z]*)(t)([^a-z]*)(c)([^a-z]*)(h)',
            'biatch', 'bi\*\*h', 'bytch', 'b i t c h', 'b!tch', 'bi+ch', 'l3itch', 'b+i+t+c+h+'
        ],

    ' bastard ':
        [
            'ba[s|z]+t[e|a]+rd', 'b+a+s+t+a+r+d+'
        ],

    ' trans gender':
        [
            'transgender'
        ],

    ' gay ':
        [
            'gay', 'g+a+y+'
        ],

    ' cock ':
        [
            '[^a-z]cock', 'c0ck', '[^a-z]cok ', 'c0k', '[^a-z]cok[^aeiou]', ' cawk',
            '(c)([^a-z ])(o)([^a-z ]*)(c)([^a-z ]*)(k)', 'c+o+c+k+'
        ],

    ' dick ':
        [
            ' dick[^aeiou]', 'deek', 'd i c k', 'dik', 'd+i+c+k+'
        ],

    ' suck ':
        [
            'sucker', 's+u+c+k+', 'sucks', '5uck', 
        ],

    ' cunt ':
        [
            'cunt', 'c u n t'
        ],

    ' bull shit ':
        [
            'bullsh\*t', 'bull\$hit'
        ],

    ' homo sex ual':
        [
            'homosexual'
        ],

    ' jerk ':
        [
            'jerk'
        ],

    ' idiot ':
        [
            'i[d]+io[t]+', '(i)([^a-z ]*)(d)([^a-z ]*)(i)([^a-z ]*)(o)([^a-z ]*)(t)', 'idiots'
                                                            
        ],

    ' dumb ':
        [
            '(d)([^a-z ]*)(u)([^a-z ]*)(m)([^a-z ]*)(b)'
        ],

    ' shit ':
        [
            'shitty', '(s)([^a-z ]*)(h)([^a-z ]*)(i)([^a-z ]*)(t)', 'shite', '\$hit', '$h1t'
        ],

    ' shit hole ':
        [
            'shythole'
        ],

    ' retard ':
        [
            'returd', 'retad', 'retard', 'wiktard', 'wikitud'
        ],

    ' rape ':
        [
            ' raped'
        ],

    ' dumb ass':
        [
            'dumbass', 'dubass'
        ],

    ' ass head':
        [
            'butthead'
        ],

        ' sexy ':
        [
            's+e+x+y+'
        ],

    ' sex ':
        [
            's+e+x+', 's3x', 'sexuality'
        ],

    # ' nigga ':
    #     [
            
    #     ],

    ' nigger ':
        [
            'nigger', ' nigr ', 'negrito', 'niguh', 'n3gr', 'n i g g e r', 'nigga', 'nig+a', 'n+i+g+a+'
        ],


    ' shut the fuck up ':
        [
            'stfu', 'st+u', 'shut the fuck up', 'shut the f\*ck up', 'sh\*t the f\*ck up'
        ],

    ' pussy ':
        [
            'pussy[^c]', 'pusy', 'pussi[^l]', 'pusses', 'p*ssy'
        ],

    ' faggot ':
        [
            'faggot', ' fa[g]+[s]*[^a-z ]', 'fagot', 'faggit',
            '(f)([^a-z ]*)(a)([^a-z ]*)([g]+)([^a-z ]*)(o)([^a-z ]*)(t)', 'fau[g]+ot', 'fae[g]+ot',
        ],

    ' mother fucker':
        [
            ' motha ', ' motha f', ' mother f', 'motherucker',
        ],

    ' whore ':
        [
            'wh\*\*\*', 'w h o r e'
        ],
    ' fucking ':
        [
            'f*$%-ing'
        ],
}


In [5]:
def mask(message):
  ans = ""
  for i in range(0, len(message)):
    if i%2==1:
      ans += '*'
    else:
      ans += message[i]
  # print("ans "+ans)
  return ans

In [11]:
def clean_text(text,remove_repeat_text=True, remove_patterns_text=True, is_lower=True):
  modified = text
  oldcomm = ""

  if is_lower:
    text=text.lower()
  if remove_patterns_text:
    for target, patterns in RE_PATTERNS.items():
      for pat in patterns:
        text = re.sub(pat, target, text)
        # text = str(text).replace(pat, target)
      text = str(text).replace(str(target), target)

    for word in text.split(' '):
      if word==" ":
        continue
      word_new = " "+word+" "
      if word_new in RE_PATTERNS.keys() or word_new in RE_PATTERNS.values():
        oldcomm += mask(word)+" "
      else:
        oldcomm += word+" "
  else:
    oldcomm = text

  if remove_repeat_text:
    text = re.sub(r'(.)\1{2,}', r'\1', text) 

  text = str(text).replace("\n", " ")
  text = re.sub(r'[^\w\s]',' ',text)
  text = re.sub('[0-9]',"",text)
  text = re.sub(" +", " ", text)
  text = re.sub("([^\x00-\x7F])+"," ",text)
  print(oldcomm)
  return text, oldcomm 

In [7]:
pickle_off = open("/content/drive/MyDrive/tokenizer.txt", "rb")
tokenizer = pickle.load(pickle_off)

In [8]:
def isToxic(prediction):
    # print(prediction[0][0])
    if prediction[0][0]>=0.45:
      return True
    elif prediction[0][0]+prediction[0][2]+prediction[0][4]>=0.80:
      return True
    else:
      return False

In [9]:
model_1 = keras.models.load_model(filepath="/content/drive/MyDrive/Models")

def toxicity_level(oldcomment, newcomment):
    new_string = [newcomment]
    new_string = tokenizer.texts_to_sequences(new_string)
    maxpadlen = 200
    new_string = pad_sequences(new_string, maxlen=maxpadlen, padding='post')
    
    prediction = model_1.predict(new_string) #(Change to model_1 or model_2 depending on the preference of model type|| Model 1: LSTM, Model 2:LSTM-CNN)
    
    toxic = isToxic(prediction)
    # print(toxic)
    if toxic==False:
      return ""

    tox = ""
    tox += "Toxicity levels for '{}':\n".format(oldcomment)
    tox += 'Toxic:         {:.0%}\n'.format(prediction[0][0])
    tox += 'Severe Toxic:  {:.0%}\n'.format(prediction[0][1])
    tox += 'Obscene:       {:.0%}\n'.format(prediction[0][2])
    tox += 'Threat:        {:.0%}\n'.format(prediction[0][3])
    tox += 'Insult:        {:.0%}\n'.format(prediction[0][4])
    tox += 'Identity Hate: {:.0%}\n'.format(prediction[0][5])
    
    return tox

In [10]:
apihelper.ENABLE_MIDDLEWARE = True
API_TOKEN = '1645954685:AAGxEa5-k_GxBerNuZIcwQUTd350pAkSwPg'

bot = telebot.TeleBot(API_TOKEN)

# Handle '/start' and '/help'
@bot.message_handler(commands=['start','help'])
def send_welcome(message):
    bot.reply_to(message, """\
Hi there, This is I'm seeing you bot!.\n\n\
As the name indicates, I will be seeing you..LOL\n\
Jokes a part, what I will see is the toxicity in your chats! Add me to your group\
 and give me the necessary permissions and I will keep your group clean without toxic words.\
 I will report the toxicity levels(if there is any) in your message and mask it!!.\n
Try doing that here in our private chat itself!
""")

# Handle all other messages with content_type 'text' (content_types defaults to ['text'])
@bot.message_handler(func=lambda message: True)
def echo_message(message):
    text, newtext = clean_text(message.text)
    toxicity = toxicity_level(newtext, text)
    if toxicity!="":
      bot.reply_to(message, toxicity)
      name = str(message.from_user.username)
      bot.send_message(message.chat.id, str(name)+"\n\n"+newtext)
      bot.delete_message(message.chat.id,message.message_id)


bot.polling()

True
 hello 
hello 
True
 you 
  
  
  
 bitch 
  
  
  
you    b*t*h    
True
 hey 
  
  
 bastard 
  
  
hey   b*s*a*d   
